<a href="https://www.kaggle.com/code/yogisaireddy/skincancerprediction?scriptVersionId=201879290" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import keras
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
image_size = (244, 224)
batch_size = 32

train_ds = keras.utils.image_dataset_from_directory(
    r"/kaggle/input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/train",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)
val_ds = keras.utils.image_dataset_from_directory(
    r"/kaggle/input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/test",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

In [ ]:
import numpy as np
num_classes = len(np.unique(train_ds))
print("Number of classes in the dataset:", num_classes)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    r"/kaggle/input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/train",  # Replace with your training data path
    target_size=(224, 224),
    batch_size=32,
    # Use categorical for multi-class labels
    class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    r"/kaggle/input/melanoma-skin-cancer-dataset-of-10000-images/melanoma_cancer_dataset/test",  # Replace with your validation data path
    target_size=(224, 224),
    batch_size=32,
    # Use categorical for multi-class labels
    class_mode='categorical'
)

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score,precision_score
from keras.metrics import MeanSquaredError
from keras.losses import CategoricalCrossentropy # Changed to CategoricalCrossentropy
from keras.utils import to_categorical


def evaluate_model(model, test_data, test_labels):
    # Predict the probabilities for the test dataset
    y_pred_prob = model.predict(test_data)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=-1)

    # One-hot encode test_labels
    #test_labels = to_categorical(test_labels, num_classes=9) # This line is not needed as test_labels are already one-hot encoded.

    # Calculate accuracy
    accuracy = accuracy_score(np.argmax(test_labels, axis=-1), y_pred)

    # Calculate F1 score
    # Use 'weighted' average for multiclass data
    f1 = f1_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')
    precision = precision_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')

    # Calculate recall
    # Use 'weighted' average for multiclass data
    recall = recall_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')

    # Calculate categorical crossentropy loss
    loss_fn = CategoricalCrossentropy() # Changed to CategoricalCrossentropy
    loss = loss_fn(test_labels, y_pred_prob).numpy()

    return loss, accuracy, f1, recall,precision

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        label_index = labels[i].numpy().argmax()
        plt.title(label_index)  # Display the label index
        plt.axis("off")
plt.show()

In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
    for i in range(9):
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(np.array(augmented_images[0]).astype("uint8"))
        plt.axis("off")

In [ ]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(tf.image.resize(img, (224, 224))), label),
    num_parallel_calls=tf_data.AUTOTUNE,)
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

In [ ]:
import keras
from keras import layers

def skin_cancer_cnn(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Normalization
    x = layers.Rescaling(1.0 / 255)(inputs)

    # Block 1
    x = layers.Conv2D(32, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Block 2
    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Block 3
    x = layers.Conv2D(128, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Block 4
    x = layers.Conv2D(256, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Flatten and Fully Connected Layers
    x = layers.Flatten()(x)
    x = layers.Dense(512)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    x = layers.Dropout(0.5)(x)

    if num_classes == 2:
        units = 2
    else:
        units = num_classes

    outputs = layers.Dense(units, activation='softmax')(x)

    return keras.Model(inputs, outputs)

# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 2  # Assuming binary classification for skin cancer detection

# Create the model
model = skin_cancer_cnn(input_shape=input_shape, num_classes=num_classes)
model.summary()


In [ ]:
from keras import callbacks
epochs = 5

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Predict the probabilities for the test dataset using the validation_generator
y_pred_prob = model.predict(validation_generator)

# Convert probabilities to class predictions using argmax
y_pred = np.argmax(y_pred_prob, axis=-1)

# Get true labels from the generator
y_true = validation_generator.classes

# Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the ResNet50 model pre-trained on ImageNet, excluding the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(1024, activation='relu')(x)

# Add a final fully connected layer with a sigmoid activation for binary classification
predictions = Dense(2, activation='softmax')(x)  # 1 unit for binary classification

# Create the final model
model1 = Model(inputs=base_model.input, outputs=predictions)

# Optionally, freeze the layers of the base model during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary crossentropy for binary classification
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model1.summary()

In [ ]:
from keras import callbacks
epochs = 10
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model1.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model1, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict the probabilities for the test dataset
y_pred_prob = model1.predict(test_data)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# If the output is from a softmax layer with multiple classes
if y_pred.shape[-1] > 1:
    y_pred = np.argmax(y_pred, axis=-1)
    test_labels = np.argmax(test_labels, axis=-1)
else:
    y_pred = y_pred.flatten()

# Calculate confusion matrix
cm = confusion_matrix(test_labels, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the VGG-19 model with pre-trained weights on ImageNet
# Set include_top to False to exclude the final fully connected layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers to avoid retraining them
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)  # Flatten the output of the conv layers
x = Dense(256, activation='relu')(x)  # Add a fully connected layer with 256 neurons
x = Dense(128, activation='relu')(x)  # Add another fully connected layer
x = Dense(2, activation='softmax')(x)  # Output layer with 1 neuron and sigmoid activation for binary classification

# Create the new model
model3 = Model(inputs=base_model.input, outputs=x)

# Compile the model
# Use binary_crossentropy for binary classification
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model3.summary()

In [ ]:
from keras import callbacks
epochs = 10
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model3.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model3, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict the probabilities for the test dataset
y_pred_prob = model3.predict(test_data)

# Convert probabilities to binary predictions (threshold = 0.5)
y_pred = (y_pred_prob > 0.5).astype(int)

# If the output is from a softmax layer with multiple classes
if y_pred.shape[-1] > 1:
    y_pred = np.argmax(y_pred, axis=-1)
    test_labels = np.argmax(test_labels, axis=-1)
else:
    y_pred = y_pred.flatten()

# Calculate confusion matrix
cm = confusion_matrix(test_labels, y_pred)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the VGG-19 model with pre-trained weights on ImageNet
# Set include_top to False to exclude the final fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers to avoid retraining them
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)  # Flatten the output of the conv layers
x = Dense(256, activation='relu')(x)  # Add a fully connected layer with 256 neurons
x = Dense(128, activation='relu')(x)  # Add another fully connected layer
x = Dense(2, activation='softmax')(x)  # Output layer with 1 neuron and sigmoid activation for binary classification

# Create the new model
model4 = Model(inputs=base_model.input, outputs=x)

# Compile the model
# Use binary_crossentropy for binary classification
model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print a summary of the model architecture
model4.summary()

In [ ]:
from keras import callbacks
epochs = 10

model4.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model4.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:

# Use a loop to get all the batches of data
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model4, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
pip install --upgrade keras

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create a Sequential model
model5 = Sequential()

# Input layer (adjust input_shape as needed)
model5.add(Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model5.add(MaxPooling2D((2, 2)))

# Residual blocks (adjust the number of blocks and filters as desired)
model5.add(Conv2D(128, (3, 3), activation='relu'))
model5.add(Conv2D(128, (3, 3), activation='relu'))
model5.add(MaxPooling2D((2, 2)))

model5.add(Conv2D(256, (3, 3), activation='relu'))
model5.add(Conv2D(256, (3, 3), activation='relu'))
model5.add(MaxPooling2D((2, 2)))

model5.add(Conv2D(512, (3, 3), activation='relu'))
model5.add(Conv2D(512, (3, 3), activation='relu'))
model5.add(MaxPooling2D((2, 2)))

# Flatten the feature maps
model5.add(Flatten())

# Fully connected layers (adjust the number of neurons and layers as needed)
model5.add(Dense(256, activation='relu'))
model5.add(Dense(128, activation='relu'))
model5.add(Dense(2, activation='softmax'))

# Compile the model
model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
from keras import callbacks
epochs = 10

model5.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255)

callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model5.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model5, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score,precision_score
from keras.metrics import MeanSquaredError
from keras.losses import CategoricalCrossentropy # Changed to CategoricalCrossentropy
from keras.utils import to_categorical


def evaluate_model(model, test_data, test_labels):
    # Predict the probabilities for the test dataset
    y_pred_prob = model.predict(test_data)

    # Convert probabilities to class predictions
    y_pred = np.argmax(y_pred_prob, axis=-1)

    # One-hot encode test_labels
    #test_labels = to_categorical(test_labels, num_classes=9) # This line is not needed as test_labels are already one-hot encoded.

    # Calculate accuracy
    accuracy = accuracy_score(np.argmax(test_labels, axis=-1), y_pred)

    # Calculate F1 score
    # Use 'weighted' average for multiclass data
    f1 = f1_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')
    precision = precision_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')

    # Calculate recall
    # Use 'weighted' average for multiclass data
    recall = recall_score(np.argmax(test_labels, axis=-1), y_pred, average='weighted')

    # Calculate categorical crossentropy loss
    loss_fn = CategoricalCrossentropy() # Changed to CategoricalCrossentropy
    loss = loss_fn(test_labels, y_pred_prob).numpy()

    return loss, accuracy, f1, recall,precision

In [ ]:
import keras
from keras import layers

def skin_cancer_cnn(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Normalization
    x = layers.Rescaling(1.0 / 255)(inputs)

    # Block 1
    x = layers.Conv2D(32, 4, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("sigmoid")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Block 2
    x = layers.Conv2D(64, 4, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("sigmoid")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.3)(x)

    # Block 3
    x = layers.Conv2D(128, 4, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("sigmoid")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Block 4
    x = layers.Conv2D(256, 4, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("sigmoid")(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Dropout(0.4)(x)

    # Flatten and Fully Connected Layers
    x = layers.Flatten()(x)
    x = layers.Dense(512)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("sigmoid")(x)
    x = layers.Dropout(0.5)(x)

    if num_classes == 2:
        units = 2
    else:
        units = num_classes

    outputs = layers.Dense(units, activation='softmax')(x)

    return keras.Model(inputs, outputs)

# Define the input shape and number of classes
input_shape = (224, 224, 3)
num_classes = 2  # Assuming binary classification for skin cancer detection

# Create the model
model1 = skin_cancer_cnn(input_shape=input_shape, num_classes=num_classes)
model1.summary()


In [ ]:
from keras import callbacks
epochs = 10

# Use categorical_crossentropy for multi-class problems

model1.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=4e-2),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model1.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model1, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load the ResNet50 model pre-trained on ImageNet, excluding the top fully connected layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add global average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer with 1024 units and ReLU activation
x = Dense(256, activation='sigmoid')(x)

# Add a final fully connected layer with a sigmoid activation for binary classification
predictions = Dense(2, activation='softmax')(x)  # 1 unit for binary classification

# Create the final model
model2 = Model(inputs=base_model.input, outputs=predictions)

# Optionally, freeze the layers of the base model during training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model with binary crossentropy for binary classification
model2.compile(optimizer='RMSprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model2.summary()

In [ ]:
from keras import callbacks
epochs = 15

model2.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=4e-2),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model2.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model2, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the VGG-19 model with pre-trained weights on ImageNet
# Set include_top to False to exclude the final fully connected layers
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers to avoid retraining them
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)  # Flatten the output of the conv layers
x = Dense(1028, activation='sigmoid')(x)  # Add a fully connected layer with 256 neurons
x = Dense(128, activation='sigmoid')(x)  # Add another fully connected layer
x = Dense(2, activation='softmax')(x)  # Output layer with 1 neuron and sigmoid activation for binary classification

# Create the new model
model3 = Model(inputs=base_model.input, outputs=x)

# Print a summary of the model architecture
model3.summary()

In [ ]:
from keras import callbacks
epochs = 15

model3.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=4e-2),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model3.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model3, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# Load the VGG-19 model with pre-trained weights on ImageNet
# Set include_top to False to exclude the final fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the convolutional layers to avoid retraining them
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top of the base model
x = Flatten()(base_model.output)  # Flatten the output of the conv layers
x = Dense(1028, activation='sigmoid')(x)  # Add a fully connected layer with 256 neurons
x = Dense(128, activation='sigmoid')(x)  # Add another fully connected layer
x = Dense(2, activation='softmax')(x)  # Output layer with 1 neuron and sigmoid activation for binary classification

# Create the new model
model4 = Model(inputs=base_model.input, outputs=x)

# Print a summary of the model architecture
model4.summary()

In [ ]:
from keras import callbacks
epochs = 15

model4.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=4e-2),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model4.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model4, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Create a Sequential model
model5 = Sequential()

# Input layer (adjust input_shape as needed)
model5.add(Conv2D(64, (4, 4), activation='sigmoid', input_shape=(224, 224, 3)))
model5.add(MaxPooling2D((3, 3)))

# Residual blocks (adjust the number of blocks and filters as desired)
model5.add(Conv2D(128, (4, 4), activation='sigmoid'))
model5.add(Conv2D(128, (4, 4), activation='sigmoid')
model5.add(MaxPooling2D((3, 3)))

model5.add(Conv2D(256, (4, 4), activation='sigmoid'))
model5.add(Conv2D(256, (4, 4), activation='sigmoid')
model5.add(MaxPooling2D((3, 3)))

model5.add(Conv2D(512, (4, 4), activation='sigmoid'))
model5.add(Conv2D(512, (4, 4), activation='sigmoid'))
model5.add(MaxPooling2D((3, 3)))

# Flatten the feature maps
model5.add(Flatten())

# Fully connected layers (adjust the number of neurons and layers as needed)
model5.add(Dense(256, activation='sigmoid'))
model5.add(Dense(128, activation='sigmoid'))
model5.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
from keras import callbacks
epochs = 15

model5.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=4e-2),
    # Use categorical crossentropy for multi-class problems
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.CategoricalAccuracy(name="accuracy")],
)
callbacks_list = [
    callbacks.ModelCheckpoint("best_model.keras", monitor="val_loss", save_best_only=True, mode="min"),
    callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min"),
]
history = model5.fit(
    train_generator,
    epochs=epochs,
    callbacks=callbacks_list,
    validation_data=validation_generator
)

In [ ]:
test_data = []
test_labels = []
for _ in range(len(validation_generator)): # Loop through the entire generator
  batch_x, batch_y =next(validation_generator)
  test_data.append(batch_x)
  test_labels.append(batch_y)

# Concatenate all batches into a single array
test_data = np.concatenate(test_data, axis=0)
test_labels = np.concatenate(test_labels, axis=0)

# Assuming you have test_data and test_labels ready
loss, accuracy, f1, recall,precision = evaluate_model(model5, test_data, test_labels)
print(f'Loss: {loss:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')